# Hierarchical, Global, and Panel Forecasting with `sktime`

### Overview of this notebook

* introduction - hierarchical time series
* hierarchical and panel data format in sktime
* automated vectorization of forecasters and transformers
* hierarchical reconciliation in sktime
* hierarchical/global forecasting using summarization/reduction models, "M5 winner"
* extender guide
* contributor credits

In [ ]:
import warnings

warnings.filterwarnings("ignore")

---
## Hierarchical time series

Time series are often present in "hierarchical" form, example:

<img src="./img/hierarchytree.png" width="1200" alt="arrow heads">

Examples include:
* Product sales in different categories (e.g. M5 time series competition)
* Tourism demand in different regions
* Balance sheet structures across cost centers / accounts

Many hierarchical time series datasets can be found here:
https://forecastingdata.org/
(access loader in development)

For literature see also:
https://otexts.com/fpp2/hierarchical.html

A time series can also exhibit multiple independent hierarchies:

<img src="./img/hierarchytree_grouped.png" width="1200" alt="arrow heads">

---

### Representation of hierarchical and panel datatypes

`sktime` distinguishes as abstract scientific data types for time series (="scitypes"):

* `Series` type = one time series (uni or multivariate)
* `Panel` type = collection of multiple time series, flat hierarchy
* `Hierarchical` type = hierarchical collection of time series, as above

Each scitype has multiple mtype representations.

For simplicitly, we focus only on `pandas.DataFrame` based representations below.

In [ ]:
# import to retrieve examples

from sktime.datatypes import get_examples

#### Individual time series - `Series` scitype, `"pd.DataFrame"` mtype

In the `"pd.DataFrame"` mtype, time series are represented by an in-memory container `obj: pandas.DataFrame` as follows.

* structure convention: `obj.index` must be monotonous, and one of `Int64Index`, `RangeIndex`, `DatetimeIndex`, `PeriodIndex`.
* variables: columns of `obj` correspond to different variables
* variable names: column names `obj.columns`
* time points: rows of `obj` correspond to different, distinct time points
* time index: `obj.index` is interpreted as a time index.
* capabilities: can represent multivariate series; can represent unequally spaced series

Example of a univariate series in `"pd.DataFrame"` representation.\
This series has two variables, named `"a"` and `"b"`. Both are observed at the same four time points 0, 1, 2, 3.

In [ ]:
get_examples(mtype="pd.DataFrame")[0]

Example of a bivariate series in `"pd.DataFrame"` representation.\
This series has two variables, named `"a"` and `"b"`. Both are observed at the same four time points 0, 1, 2, 3.

In [ ]:
get_examples(mtype="pd.DataFrame")[1]

#### Panels (= flat collections) of time series - `Panel` scitype, `"pd-multiindex"` mtype

In the `"pd-multiindex"` mtype, time series panels are represented by an in-memory container `obj: pandas.DataFrame` as follows.

* structure convention: `obj.index` must be a pair multi-index of type `(RangeIndex, t)`, where `t` is one of `Int64Index`, `RangeIndex`, `DatetimeIndex`, `PeriodIndex` and monotonous.
* instances: rows with the same `"instances"` index correspond to the same instance; rows with different `"instances"` index correspond to different instances.
* instance index: the first element of pairs in `obj.index` is interpreted as an instance index. 
* variables: columns of `obj` correspond to different variables
* variable names: column names `obj.columns`
* time points: rows of `obj` with the same `"timepoints"` index correspond correspond to the same time point; rows of `obj` with different `"timepoints"` index correspond correspond to the different time points.
* time index: the second element of pairs in `obj.index` is interpreted as a time index. 
* capabilities: can represent panels of multivariate series; can represent unequally spaced series; can represent panels of unequally supported series; cannot represent panels of series with different sets of variables.

Example of a panel of multivariate series in `"pd-multiindex"` mtype representation.
The panel contains three multivariate series, with instance indices 0, 1, 2. All series have two variables with names `"var_0"`, `"var_1"`. All series are observed at three time points 0, 1, 2.

In [ ]:
get_examples(mtype="pd-multiindex")[0]

#### Hierarchical time series - `Hierarchical` scitype, `"pd_multiindex_hier"` mtype

* structure convention: `obj.index` must be a 3 or more level multi-index of type `(RangeIndex, ..., RangeIndex, t)`, where `t` is one of `Int64Index`, `RangeIndex`, `DatetimeIndex`, `PeriodIndex` and monotonous. We call the last index the "time-like" index
* hierarchy level: rows with the same non-time-like indices correspond to the same hierarchy unit; rows with different non-time-like index combination correspond to different hierarchy unit.
* hierarchy: the non-time-like indices in `obj.index` are interpreted as a hierarchy identifying index. 
* variables: columns of `obj` correspond to different variables
* variable names: column names `obj.columns`
* time points: rows of `obj` with the same `"timepoints"` index correspond correspond to the same time point; rows of `obj` with different `"timepoints"` index correspond correspond to the different time points.
* time index: the last element of tuples in `obj.index` is interpreted as a time index. 
* capabilities: can represent hierarchical series; can represent unequally spaced series; can represent unequally supported hierarchical series; cannot represent hierarchical series with different sets of variables.

In [ ]:
X = get_examples(mtype="pd_multiindex_hier")[0]
X

In [ ]:
X.index.get_level_values(level=-1)

---

### Automated vectorization (casting) of forecasters and transformers

Many transformers and forecasters implemented for single series

`sktime` automatically vectorizes/"up-casts" them to hierarchical data

"apply by individual series/panel in the hierarchical structure"

constructing a hierarchical time series:

In [ ]:
from sktime.utils._testing.hierarchical import _make_hierarchical

y = _make_hierarchical()
y

all forecasters and transformers are applicable to this!\
if forecaster implements logic for single series only, "apply to all hierarchy units"

In [ ]:
from sktime.forecasting.arima import ARIMA

forecaster = ARIMA()

y_pred = forecaster.fit(y, fh=[1, 2]).predict()
y_pred

individual forecasters fitted per hierarchy level are stored in the  `forecasters_` attribute\
this contains a `pandas.DataFrame` in the same format as the hierarchy levels, and fitted forecasters inside\
(for transformers, the attribute is called `transformers_`)

In [ ]:
forecaster.forecasters_

to access or inspect individual forecasters, access elements of the `forecasters_` data frame:

In [ ]:
forecaster.forecasters_.iloc[0, 0].summary()

auto-vectorization is also performed for probabilistic forecasts (see notebook 2):

In [ ]:
forecaster.predict_interval()

which level a forecaster/natively implements and when vectorization occurs is determined by the "inner mtype" attributes.

For forecasters, inspect `y_inner_mtype`, this is a list of internally supported machine types:

In [ ]:
forecaster.get_tag("y_inner_mtype")

ARIMA supports only one `Series` level mtype.
For a register of all mtype codes and their corresponding scitype (series, panel or hierarchical),\
see `sktime.datatypes.MTYPE_REGISTER` (convert to `pandas.DataFrame` for pretty-printing)

In [ ]:
import pandas as pd

from sktime.datatypes import MTYPE_REGISTER

pd.DataFrame(MTYPE_REGISTER)

roadmap items:

* interfacing and implementing common methods with native hierarchical support
    * ARIMA using hierarchy factors, GEE, mixed effects
* wrappers for "using hierarchy levels" as covariates or exogeneous features
* full vectorization over variables to render univariate forecasters multivariate

contributions are appreciated!

---

### Hierarchical reconciliation

#### Why do we need hierarchical reconciliation

forecast reconciliation = ensuring that linear hierarchy dependencies are met,\
e.g., "sum of individual shop sales in Berlin must equal sum of total sales in Berlin"\
requires hierarchical (or panel) data, usually involves totals

**Bottom up reconciliation** works by producing only forecasts at the lowest level and then adding up to totals across all higher levels. 

    * Arguably the most simple of all algorithms to reconcile across hierarchies.
    * Advantages: easy to implement
    * Disadvantages: lower levels of hierarchy are prone to excess volatility. This excess volatility is aggregated up, often producing much less accurate top level forecasts.

**Top down reconciliation** works by producing top level forecasts and breaking them down to  the lowest level based e.g. on relative proportions of those lower levels.

    * Advantages: still easy to implement, top level is stable
    * Disadvantages: peculiarities of lower levels of hierarchy ignored

**Optimal forecast reconciliation** works by producing forecasts at all levels of the hierarchy and adjusting all of them in a way that seeks to minimize the forecast errors

    * Advantages: often found to be most accurate implementation
    * Disadvantages: more difficult to implement

`sktime` provides functionality for reconciliation:

* data container convention for node-wise aggregates
* functionality to compute node-wise aggregates - `Aggregator`
    * can be used for bottom-up reconciliation
* transformer implementing reconiliation logic - `Reconciler`
    * implements top-down reconciliation
    * implements transformer-like optimal forecast reconciliation

#### The node-wise aggregate data format

`sktime` uses a special case of the `pd_multiindex_hier` format to store node-wise aggregates:

* a `__total` index element in an instance (non-time-like) level indicates summation over all instances below that level
* the `__total` index element is reserved and cannot be used for anything else
* entries below a `__total` index element are sums of entries over all other instances in the same levels where a `__total` element is found

#### The aggregation transformer

The node-wise aggregated format can be obtained by applying the `Aggregator` transformer.

In a pipeline with non-aggregate dinput, this allows making forecasts by totals.

In [ ]:
from sktime.datatypes import get_examples

y_hier = get_examples("pd_multiindex_hier")[1]
y_hier

In [ ]:
from sktime.transformations.hierarchical.aggregate import Aggregator

Aggregator().fit_transform(y_hier)

If used at the start of a pipeline, forecasts are made for node `__total`-s as well as individual instances.

Note: in general, this does not result in a reconciled forecast, i.e., forecast totals will not add up.

In [ ]:
from sktime.forecasting.naive import NaiveForecaster

pipeline_to_forecast_totals = Aggregator() * NaiveForecaster()

pipeline_to_forecast_totals.fit(y_hier, fh=[1, 2])
pipeline_to_forecast_totals.predict()

If used at the end of a pipeline, forecasts are reconciled bottom-up.

That will result in a reconciled forecast, although bottom-up may not be the method of choice.

In [ ]:
from sktime.forecasting.naive import NaiveForecaster

pipeline_to_forecast_totals = NaiveForecaster() * Aggregator()

pipeline_to_forecast_totals.fit(y_hier, fh=[1, 2])
pipeline_to_forecast_totals.predict()

#### Advanced reconciliation

For transformer-like reconciliation, use the `Reconciler`.
It supports advanced techniques such as OLS and WLS:

In [ ]:
from sktime.transformations.hierarchical.reconcile import Reconciler

pipeline_with_reconciliation = (
    Aggregator() * NaiveForecaster() * Reconciler(method="ols")
)

In [ ]:
pipeline_to_forecast_totals.fit(y_hier, fh=[1, 2])
pipeline_to_forecast_totals.predict()

Roadmap items:

* reconciliation of wrapper type
* reconciliation & global forecasting
* probabilistic reconciliation

---

### Global/panel forecasting - introduction

Global forecasting = training **across sets of time series**, i.e., on time series **panels**.\
Typically better than "fit one forecaster per time series instance".

Also called "panel forecasting" for homogeneous/contemporaneous index sets.

 Why does global forecasting matter?
 * In practice, we often have time series of limited range
 * Estimation is difficult, and we cannot model complex dependencies
 * Assumption of global forecasting: We can observe the identical data generating process (DGP) multiple times
 * Non-identical DGPs can be fine too, as long as the degree of dissimilarity is captured by exogeneous information
 * Now we have much more information and can estimate more reliably and more complex models (caveat: unless complexity is purely driven by time dynamics)
 
As a result of these advantages, global forecasting models have been very successful in competition, e.g.
* Rossmann Store Sales
* Walmart Sales in Stormy Weather
* M5 competition

Many business problems in practice are essentially global forecasting problem - often also reflecting hierarchical information (see above)
* Product sales in different categories (e.g. M5 time series competition)
* Balance sheet structures across cost centers / accounts
* Dynamics of pandemics observed at different points in time

Distinction to multivariate forecasting
* Multivariate forecasting focuses on modeling interdependence between time series
* Global can model interdependence, but focus lies on enhancing observation space

Implementation in sktime
* Multivariate forecasting models are supported in sktime via ? VAR...* 
* Global forecasting

For the following example we will use the `"pd-multiindex"` representation of the `"Panel"` scitype (see above)

Row multiindex level 0 is the unique identifier for the individual time series, level 1 contains the time index.

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline

from sktime.forecasting.compose import make_reduction
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.transformations.series.date import DateTimeFeatures

pd.options.mode.chained_assignment = None
pd.set_option("display.max_columns", None)

# %%
# Load M5 Data and prepare
y = pd.read_pickle("global_fc/y.pkl")
X = pd.read_pickle("global_fc/X.pkl")

`y`/`X` are based on the M5 competition.
The data features sales of products in different stores, different states and different product categories. 

For a detailed analysis of the competition please take a look at the paper 
"M5 accuracy competition: Results, findings, and conclusions".

https://doi.org/10.1016/j.ijforecast.2021.11.013

You can see 
a glimpse of the data here:

In [ ]:
print(y.head())
print(X.head())


* time series grouped via the instances argument in the first column = Panel
* focus on modeling individual products
* hierarchical information is provided as exgoneous information. 

For the M5 competition, winning solution used exogeneous features about the hierarchies like `"dept_id"`, `"store_id"` etc. to capture similarities and dissimilarities of the products. Other features include holiday events and snap days (specific assisstance program of US social security paid on certain days).

now split into test and train set using `temporal_train_test_split`.\
we can cut every instance of the time series individually:


In [ ]:
y_train, y_test, X_train, X_test = temporal_train_test_split(y, X)
print(y_train.head(5))
print(y_test.head(5))

both `y` and `X` are split in the same way, and hierarchy structures are preserved.

### Rationale for tree based models

Tree ensembles exploit complex non-linear relationships / dependencies between time series and covariates.

In **univariate time series forecasting**, tree based models often do not perform well due to lack of data.

Due to large effective sample sizes in **global forecasting** , tree ensembles can become a good choice (e.g., 42,840 time series in the M5 competition).

sktime can interface any sklearn compatible model via reduction, e.g., `RandomForestRegressor`.

In [ ]:
regressor = make_pipeline(
    RandomForestRegressor(random_state=1),
)

Caveat: reduction applies a supervised regressor to a time series, i.e., to a task for which they were not originally designed.

This issue can be addressed by generating features that capture the dynamics of the time series.\
`"WindowSummarizer"` can be used to generate features useful for time series forecasting,\
 based on a provided dictionary of functions, window shifts and window lengths:

In [ ]:
import pandas as pd

from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.compose import ForecastingPipeline
from sktime.transformations.series.summarize import WindowSummarizer

kwargs = {
    "lag_feature": {
        "lag": [1],
        "mean": [[1, 3], [3, 6]],
        "std": [[1, 4]],
    }
}

transformer = WindowSummarizer(**kwargs)
y_transformed = transformer.fit_transform(y_train)

y_transformed.head(10)

The notation `"mean": [[1, 3]]` (captured in the column `"y_mean_1_3"`) is read as:

summarization function `"mean": [[1, 3]]` is applied to:

* window of length 3
* start (inclusive) is lagged by one period

Visualization:

For z = target time stamp:\
`window = [1, 3]`, means a `lag` of 1 and `window_length` of 3, selecting the three last days (exclusive z).

Summarization is done across windows like this:

| x x x x x x x x * * * z x |
|---------------------------|

By default, `"WindowSummarizer"` uses pandas rolling window functions to allow for a speedy generation of features. 
* "sum",
* "mean",
* "median",
* "std",
* "var",
* "kurt",
* "min",
* "max",
* "corr",
* "cov",
* "skew",
* "sem"

typically very fast since optimized for rolling, grouped operations

In the M5 competition, arguably the most relevant features were:

* **mean** calculations to capture level shifts, e.g. last week sales, sales of the week prior to the last month etc.
* **standard deviation** to capture increases / decreases in volatility in sales, and how it impacts future sales
* rolling **skewness** / **kurtosis** calculations, to capture changes in store sales tendencies.
* various different calculations to capture periods of zero sales (e.g. out of stock scenarios)

First three are available via native pandas functions.\
`WindowSummarizer` can also be provided with arbitrary summarizer functions.

Example: function `count_gt130` to count how many observations lie above the threshold of 130 within a window of length 3, lagged by 2 periods.

In [ ]:
import numpy as np


def count_gt130(x):
    """Count how many observations lie above threshold 130."""
    return np.sum((x > 700)[::-1])


kwargs = {
    "lag_feature": {
        "lag": [1],
        count_gt130: [[2, 3]],
        "std": [[1, 4]],
    }
}

transformer = WindowSummarizer(**kwargs)
y_transformed = transformer.fit_transform(y_train)

y_transformed.head(10)

Above applies `"WindowSummarizer"` to the forecasting targey `y`.

To apply `"WindowSummarizer"` to columns in `X`, use `"WindowSummarizer"` within a `"ForecastingPipeline"` and specify `"target_cols"`.

In the M5 competition, lagging of exogeneous features was especially useful for lags around holiday dummies (often sales are affected for a few days before and after major holidays) as well as changes in item prices (discounts as well as persistent price changes)

In [ ]:
from sktime.datasets import load_longley
from sktime.forecasting.naive import NaiveForecaster

y_ll, X_ll = load_longley()
y_train_ll, y_test_ll, X_train_ll, X_test_ll = temporal_train_test_split(y_ll, X_ll)
fh = ForecastingHorizon(X_test_ll.index, is_relative=False)
# Example transforming only X
pipe = ForecastingPipeline(
    steps=[
        ("a", WindowSummarizer(n_jobs=1, target_cols=["POP", "GNPDEFL"])),
        ("b", WindowSummarizer(n_jobs=1, target_cols=["GNP"], **kwargs)),
        ("forecaster", NaiveForecaster(strategy="drift")),
    ]
)
pipe_return = pipe.fit(y_train_ll, X_train_ll)
y_pred1 = pipe_return.predict(fh=fh, X=X_test_ll)

y_pred1

For efficiency, relevant features are computed in a parallel way.\
For maximum parallelization, pass `WindowSummarizer` as a single transformer within `make_reduction`. 

In this case, `window_length` is inferred from the `WindowSummarizer` and need not be passed to `make_reduction`.

In [ ]:
forecaster = make_reduction(
    regressor,
    scitype="tabular-regressor",
    transformers=[WindowSummarizer(**kwargs, n_jobs=1)],
    window_length=None,
    strategy="recursive",
    pooling="global",
)

Concepts relating to calendar seasonalities need to be provided by means of feature engineering to most models. Examples:

* day of the week effects historically observed for stock prices (prices on Fridays used to differ from Monday prices).
* used car prices being higher in spring than in summer
* spendings at the beginning of the month differing from end of month due to salary effects.

Calendar seasonalities can be modeled by means of dummy variables or fourier terms. As a rule of thumb, use dummy variables for discontinous effects and fourier/period/seasonality terms when you believe there is a certain degree of smoothness in the seasonality.

`sktime` supports calendar dummy features via the `DateTimeFeatures` transformer. Manually specify the desired seasonality, or provide base frequency of the time series (daily, weekly etc.) and the desired complexity (few vs many features), `DateTimeFeatures` can infer sensible seasonality.

In [ ]:
transformer = DateTimeFeatures(ts_freq="D")
X_hat = transformer.fit_transform(X_train)

new_cols = [i for i in X_hat if i not in X_train.columns]
X_hat[new_cols]

DateTimeFeatures supports the following frequencies:
* Y - year
* Q - quarter
* M - month
* W - week
* D - day
* H - hour
* T - minute
* S - second
* L - millisecond

You can specify the manual generation of dummy features with the notation e.g. "day_of_month", "day_of_week", "week_of_quarter". 

In [ ]:
transformer = DateTimeFeatures(manual_selection=["week_of_month", "day_of_quarter"])
X_hat = transformer.fit_transform(X_train)

new_cols = [i for i in X_hat if i not in X_train.columns]
X_hat[new_cols]

### Putting it all together

Using the `"WindowSummarizer"`, `"DateTimeFeatures"` and the `"make_reduction"` function we can now set up a working example of a an end to end global forecasting pipeline based on a sample of the M5 competition data:

In [ ]:
pipe = ForecastingPipeline(
    steps=[
        (
            "event_dynamics",
            WindowSummarizer(
                n_jobs=-1, **kwargs, target_cols=["event_type_1", "event_type_2"]
            ),
        ),
        ("snap_dynamics", WindowSummarizer(n_jobs=-1, target_cols=["snap"])),
        ("daily_season", DateTimeFeatures(ts_freq="D")),
        ("forecaster", forecaster),
    ]
)

pipe_return = pipe.fit(y_train, X_train)
y_pred1 = pipe_return.predict(fh=1, X=X_test)

y_pred1

---
## Building your own hierarchical forecaster

Getting started:

* follow the ["implementing estimator" developer guide](https://www.sktime.org/en/stable/developer_guide/add_estimators.html)
* use the advanced [forecasting extension template](https://github.com/alan-turing-institute/sktime/blob/main/extension_templates/forecasting.py)

Extension template = python "fill-in" template with to-do blocks that allow you to implement your own, sktime-compatible forecasting algorithm.

Check estimators using `check_estimator`

For hierarchical forecasting:

* ensure to pick supported mtypes for panel and hierarchical data
* recommended: `y_inner_mtype = ["pd.DataFrame", "pd-multiindex", "pd_multiindex_hier"]`, same for `X_inner_mtype`
    * this ensures the inputs `y`, `X` seen in `_fit`, `_predict` are `pd.DataFrame`, with 1, 2, 3 or more row levels
* you can implement vectorization over rows if efficient implementation is available
    * but: automated vectorization already loops over row index sets, don't implement that if that's what "hierarchical" is
    * to ensure automated vectorization, do *not* include Hierarchical or Panel mtypes in `y_inner_mtype`, `X_inner_mtype`
* think carefully whether your estimator is a forecaster, or can be decomposed in a transformer
    * "do X and then apply forecaster already in sktime" is a strong hint that you actually want to implement a transformer


---

### Credits

notebook creation: danbartl, fkiraly

hierarchical forecasting framework: ciaran-g, fkiraly\
reduction compatibility with hierarchical forecasting: danbartl\
window summarizer, reduction with transform-from-y: danbartl\
aggregation and reconciliation: ciaran-g